# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [ ]:
from __future__ import print_function
import SimpleITK as sitk
import six
import numpy as np
import os  # needed navigate the system to get the input data
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

##### Extração das features

In [ ]:
# Inicialização do extractor
extractor = featureextractor.RadiomicsFeatureExtractor()

# Inicialização da builder dataframe
df_builder = pd.DataFrame()

# Numero de imagens que não foram usadas
not_used = 0

# Iterar sobre as imagens
for imagem in os.listdir('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/originalimages'):
    for mascara in os.listdir('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/maskimages'):
        if imagem == mascara:
            image_1 = Image.open('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/originalimages/' + imagem)
            label_1 = Image.open('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/maskimages/' + mascara)

            image_1 = np.asarray(image_1)
            image_1 = sitk.GetImageFromArray(image_1)

            pixels = label_1.load() # create the pixel map

            for i in range(label_1.size[0]): # for every pixel:
                for j in range(label_1.size[1]):
                    if pixels[i,j] != (0, 0, 0, 255):
                        #mudar para branco se mao for preto
                        #temos que fazer isto para ter apenas uma label
                        pixels[i,j] = (255, 255 ,255, 255)

            label_1.save('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/LIDC-IDRI-alterado.png')

            # Extração das features ---------------------------------------------
            image_1 = sitk.ReadImage('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/originalimages/' + imagem)
            label_1 = sitk.ReadImage('C:/Users/Guilherme/Desktop/Faculdade/Terceiroano/LabIACD/LIDC-IDRI-alterado.png')

            image_1 = np.asarray(image_1)
            image_1 = sitk.GetImageFromArray(image_1)

            label_1 = np.asarray(label_1)
            label_1 = sitk.GetImageFromArray(label_1)

            # Existem imagens que não podem ser usadas
            try:
                result = extractor.execute(image_1, label_1, label=255)
            except:
                print("Mask has to few dimensions, skipping")
                continue

            # Colocar na dataframe
            if df_builder.empty:
                id_list = ['id']
                df_builder = pd.concat([df_builder, pd.DataFrame(id_list + list(result.keys()))], axis=1)
            
            #tirar o png do nome da imagem
            imagem = imagem.split('.')[0]
            img_list=[imagem]
            df_builder = pd.concat([df_builder, pd.DataFrame(img_list + list(result.values()))], axis=1)


In [ ]:
df_builder.head(20)

##### Guardar os resultados num ficheiro excell

In [ ]:
#Transpor a informação da dataframe
df_builder.T.reset_index()

df_final = df_builder.transpose()

# Exportar a dataframe para excel
df_final.to_excel('pyradiomics.xlsx', header=False)